# <center>Домашнее задание № 10 (демо)
## <center> Прогнозирование задержек вылетов

Ваша задача – побить единственный бенчмарк в [соревновании](https://www.kaggle.com/c/flight-delays-2017) на Kaggle Inclass. Подробных инструкций не будет, будет только тезисно описано, как получен этот бенчмарк. Конечно, с помощью Xgboost. Надеюсь, на данном этапе курса вам достаточно бросить полтора взгляда на данные, чтоб понять, что это тот тип задачи, в которой затащит Xgboost. Но проверьте еще Catboost.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

In [2]:
train = pd.read_csv('flight_delays_train.csv')
test = pd.read_csv('flight_delays_test.csv')

In [3]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y


In [4]:
test.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258


Итак, надо по времени вылета самолета, коду авиакомпании-перевозчика, месту вылета и прилета и расстоянию между аэропортами вылета и прилета предсказать задержку вылета более 15 минут. В качестве простейшего бенчмарка возьмем логистическую регрессию и два признака, которые проще всего взять: `DepTime` и `Distance`. У такой модели результат – 0.68202 на LB. 

In [5]:
X_train, y_train = train[['Distance', 'DepTime']].values, train['dep_delayed_15min'].map({'Y': 1, 'N': 0}).values
X_test = test[['Distance', 'DepTime']].values

X_train_part, X_valid, y_train_part, y_valid = train_test_split(X_train, 
                                                                y_train, 
                                                                test_size=0.3, 
                                                                random_state=17)

In [6]:
logit = LogisticRegression(random_state=17, solver='lbfgs')

logit.fit(X_train_part, y_train_part)
logit_valid_pred = logit.predict_proba(X_valid)[:, 1]

roc_auc_score(y_valid, logit_valid_pred)

0.6795697123357751

In [9]:
logit.fit(X_train, y_train)
logit_test_pred = logit.predict_proba(X_test)[:, 1]

pd.Series(logit_test_pred, 
          name='dep_delayed_15min').to_csv('submit_logit_2feat.csv', 
                                           index_label='id', 
                                           header=True)

Как был получен бенчмарк в соревновании:
- Признаки `Distance` и  `DepTime` брались без изменений
- Создан признак "маршрут" из исходных `Origin` и `Dest`
- К признакам `Month`, `DayofMonth`, `DayOfWeek`, `UniqueCarrier` и "маршрут" применено OHE-преобразование (`LabelBinarizer`)
- Выделена отложенная выборка
- Обучалась логистическая регрессия и градиентный бустинг (xgboost), гиперпараметры бустинга настраивались на кросс-валидации, сначала те, что отвечают за сложность модели, затем число деревьев фиксировалось равным 500 и настраивался шаг градиентного спуска
- С помощью `cross_val_predict` делались прогнозы обеих моделей на кросс-валидации (именно предсказанные вероятности), настраивалась линейная смесь ответов логистической регрессии и градиентного бустинга вида $w_1 * p_{logit} + (1 - w_1) * p_{xgb}$, где $p_{logit}$ – предсказанные логистической регрессией вероятности класса 1, $p_{xgb}$ – аналогично. Вес $w_1$ подбирался вручную. 
- В качестве ответа для тестовой выборки бралась аналогичная комбинация ответов двух моделей, но уже обученных на всей обучающей выборке.

Описанный план ни к чему не обязывает – это просто то, как решение получил автор задания. Возможно, мы не захотите следовать намеченному плану, а добавите, скажем, пару хороших признаков и обучите лес из тысячи деревьев.

Удачи!

In [15]:
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold

In [16]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1)

In [17]:
ctb = CatBoostClassifier(random_seed=17)

In [21]:
train_df, y = train.drop('dep_delayed_15min', axis=1), train['dep_delayed_15min'].map({'Y': 1, 'N': 0})
train_df_part, valid_df, y_train_part, y_valid = train_test_split(train_df, y, 
                                                                  test_size=.3, stratify=y, 
                                                                  random_state=17)

In [22]:
cat_features_idx = np.where(train_df_part.dtypes == 'object')[0].tolist()

In [23]:
%%time
cv_scores = []
for train_idx, test_idx in skf.split(train_df_part, y_train_part):
    cv_train_df, cv_valid_df = train_df_part.iloc[train_idx, :], train_df_part.iloc[test_idx, :]
    y_cv_train, y_cv_valid = y_train_part.iloc[train_idx], y_train_part.iloc[test_idx]
    
    ctb.fit(cv_train_df, y_cv_train,
        cat_features=cat_features_idx);
    
    cv_scores.append(roc_auc_score(y_cv_valid, ctb.predict_proba(cv_valid_df)[:, 1]))

Learning rate set to 0.052343
0:	learn: 0.6672384	total: 465ms	remaining: 7m 44s
1:	learn: 0.6430811	total: 883ms	remaining: 7m 20s
2:	learn: 0.6232256	total: 1.23s	remaining: 6m 47s
3:	learn: 0.6049551	total: 1.62s	remaining: 6m 44s
4:	learn: 0.5878558	total: 2.37s	remaining: 7m 52s
5:	learn: 0.5727068	total: 2.69s	remaining: 7m 26s
6:	learn: 0.5610763	total: 3.04s	remaining: 7m 11s
7:	learn: 0.5490893	total: 3.39s	remaining: 7m
8:	learn: 0.5384724	total: 3.94s	remaining: 7m 13s
9:	learn: 0.5295411	total: 4.46s	remaining: 7m 21s
10:	learn: 0.5218694	total: 4.75s	remaining: 7m 7s
11:	learn: 0.5142413	total: 5.29s	remaining: 7m 15s
12:	learn: 0.5076006	total: 5.71s	remaining: 7m 13s
13:	learn: 0.5018811	total: 6.01s	remaining: 7m 3s
14:	learn: 0.4963795	total: 6.36s	remaining: 6m 57s
15:	learn: 0.4912002	total: 6.92s	remaining: 7m 5s
16:	learn: 0.4863380	total: 7.33s	remaining: 7m 3s
17:	learn: 0.4826997	total: 7.84s	remaining: 7m 8s
18:	learn: 0.4796248	total: 8.26s	remaining: 7m 6s
19

158:	learn: 0.4260721	total: 1m 27s	remaining: 7m 43s
159:	learn: 0.4260114	total: 1m 28s	remaining: 7m 42s
160:	learn: 0.4259657	total: 1m 28s	remaining: 7m 42s
161:	learn: 0.4258931	total: 1m 29s	remaining: 7m 41s
162:	learn: 0.4258235	total: 1m 29s	remaining: 7m 40s
163:	learn: 0.4257266	total: 1m 30s	remaining: 7m 39s
164:	learn: 0.4256357	total: 1m 30s	remaining: 7m 38s
165:	learn: 0.4255979	total: 1m 31s	remaining: 7m 37s
166:	learn: 0.4255120	total: 1m 31s	remaining: 7m 37s
167:	learn: 0.4254173	total: 1m 32s	remaining: 7m 35s
168:	learn: 0.4253836	total: 1m 32s	remaining: 7m 35s
169:	learn: 0.4253103	total: 1m 32s	remaining: 7m 33s
170:	learn: 0.4252517	total: 1m 33s	remaining: 7m 32s
171:	learn: 0.4251868	total: 1m 33s	remaining: 7m 31s
172:	learn: 0.4251136	total: 1m 34s	remaining: 7m 31s
173:	learn: 0.4250495	total: 1m 34s	remaining: 7m 29s
174:	learn: 0.4250245	total: 1m 35s	remaining: 7m 28s
175:	learn: 0.4249780	total: 1m 35s	remaining: 7m 27s
176:	learn: 0.4249250	total:

311:	learn: 0.4171001	total: 2m 50s	remaining: 6m 14s
312:	learn: 0.4170211	total: 2m 50s	remaining: 6m 14s
313:	learn: 0.4169761	total: 2m 51s	remaining: 6m 13s
314:	learn: 0.4168982	total: 2m 51s	remaining: 6m 13s
315:	learn: 0.4168466	total: 2m 52s	remaining: 6m 12s
316:	learn: 0.4168214	total: 2m 52s	remaining: 6m 11s
317:	learn: 0.4167381	total: 2m 53s	remaining: 6m 11s
318:	learn: 0.4166769	total: 2m 53s	remaining: 6m 10s
319:	learn: 0.4166382	total: 2m 54s	remaining: 6m 10s
320:	learn: 0.4166031	total: 2m 54s	remaining: 6m 9s
321:	learn: 0.4165469	total: 2m 55s	remaining: 6m 8s
322:	learn: 0.4164739	total: 2m 55s	remaining: 6m 7s
323:	learn: 0.4164002	total: 2m 56s	remaining: 6m 7s
324:	learn: 0.4163282	total: 2m 56s	remaining: 6m 6s
325:	learn: 0.4163039	total: 2m 56s	remaining: 6m 5s
326:	learn: 0.4161940	total: 2m 57s	remaining: 6m 5s
327:	learn: 0.4161428	total: 2m 58s	remaining: 6m 4s
328:	learn: 0.4160967	total: 2m 58s	remaining: 6m 4s
329:	learn: 0.4160462	total: 2m 59s	r

466:	learn: 0.4092819	total: 4m 8s	remaining: 4m 43s
467:	learn: 0.4092520	total: 4m 8s	remaining: 4m 42s
468:	learn: 0.4092208	total: 4m 9s	remaining: 4m 42s
469:	learn: 0.4091890	total: 4m 9s	remaining: 4m 41s
470:	learn: 0.4091322	total: 4m 10s	remaining: 4m 40s
471:	learn: 0.4090562	total: 4m 10s	remaining: 4m 40s
472:	learn: 0.4089940	total: 4m 10s	remaining: 4m 39s
473:	learn: 0.4089229	total: 4m 11s	remaining: 4m 38s
474:	learn: 0.4088745	total: 4m 11s	remaining: 4m 38s
475:	learn: 0.4088384	total: 4m 12s	remaining: 4m 37s
476:	learn: 0.4087866	total: 4m 12s	remaining: 4m 37s
477:	learn: 0.4087287	total: 4m 13s	remaining: 4m 36s
478:	learn: 0.4086924	total: 4m 14s	remaining: 4m 36s
479:	learn: 0.4086632	total: 4m 14s	remaining: 4m 35s
480:	learn: 0.4086047	total: 4m 15s	remaining: 4m 35s
481:	learn: 0.4085316	total: 4m 15s	remaining: 4m 34s
482:	learn: 0.4084775	total: 4m 16s	remaining: 4m 34s
483:	learn: 0.4084456	total: 4m 16s	remaining: 4m 33s
484:	learn: 0.4084055	total: 4m 

619:	learn: 0.4029715	total: 5m 26s	remaining: 3m 19s
620:	learn: 0.4029407	total: 5m 26s	remaining: 3m 19s
621:	learn: 0.4029197	total: 5m 27s	remaining: 3m 18s
622:	learn: 0.4028831	total: 5m 27s	remaining: 3m 18s
623:	learn: 0.4028474	total: 5m 28s	remaining: 3m 17s
624:	learn: 0.4028016	total: 5m 28s	remaining: 3m 17s
625:	learn: 0.4027428	total: 5m 29s	remaining: 3m 16s
626:	learn: 0.4027028	total: 5m 29s	remaining: 3m 16s
627:	learn: 0.4026743	total: 5m 30s	remaining: 3m 15s
628:	learn: 0.4026398	total: 5m 30s	remaining: 3m 15s
629:	learn: 0.4026141	total: 5m 31s	remaining: 3m 14s
630:	learn: 0.4025795	total: 5m 31s	remaining: 3m 14s
631:	learn: 0.4025477	total: 5m 32s	remaining: 3m 13s
632:	learn: 0.4025215	total: 5m 32s	remaining: 3m 12s
633:	learn: 0.4024614	total: 5m 33s	remaining: 3m 12s
634:	learn: 0.4024333	total: 5m 33s	remaining: 3m 11s
635:	learn: 0.4023996	total: 5m 34s	remaining: 3m 11s
636:	learn: 0.4023208	total: 5m 34s	remaining: 3m 10s
637:	learn: 0.4022899	total:

...

335:	learn: 0.4141218	total: 2m 32s	remaining: 5m 2s
336:	learn: 0.4140391	total: 2m 33s	remaining: 5m 1s
337:	learn: 0.4139973	total: 2m 33s	remaining: 5m 1s
338:	learn: 0.4139641	total: 2m 34s	remaining: 5m
339:	learn: 0.4139083	total: 2m 34s	remaining: 5m
340:	learn: 0.4138702	total: 2m 35s	remaining: 4m 59s
341:	learn: 0.4138412	total: 2m 35s	remaining: 4m 59s
342:	learn: 0.4137432	total: 2m 36s	remaining: 4m 59s
343:	learn: 0.4137066	total: 2m 36s	remaining: 4m 58s
344:	learn: 0.4136619	total: 2m 37s	remaining: 4m 58s
345:	learn: 0.4136318	total: 2m 38s	remaining: 4m 58s
346:	learn: 0.4135803	total: 2m 38s	remaining: 4m 58s
347:	learn: 0.4134899	total: 2m 39s	remaining: 4m 58s
348:	learn: 0.4134493	total: 2m 39s	remaining: 4m 57s
349:	learn: 0.4133975	total: 2m 39s	remaining: 4m 56s
350:	learn: 0.4133160	total: 2m 40s	remaining: 4m 56s
351:	learn: 0.4132680	total: 2m 40s	remaining: 4m 56s
352:	learn: 0.4132166	total: 2m 41s	remaining: 4m 55s
353:	learn: 0.4131365	total: 2m 41s	rem

489:	learn: 0.4068108	total: 3m 45s	remaining: 3m 55s
490:	learn: 0.4067616	total: 3m 46s	remaining: 3m 54s
491:	learn: 0.4067218	total: 3m 46s	remaining: 3m 54s
492:	learn: 0.4066999	total: 3m 47s	remaining: 3m 53s
493:	learn: 0.4066289	total: 3m 47s	remaining: 3m 53s
494:	learn: 0.4065853	total: 3m 47s	remaining: 3m 52s
495:	learn: 0.4065407	total: 3m 48s	remaining: 3m 52s
496:	learn: 0.4065075	total: 3m 48s	remaining: 3m 51s
497:	learn: 0.4064717	total: 3m 49s	remaining: 3m 51s
498:	learn: 0.4063732	total: 3m 49s	remaining: 3m 50s
499:	learn: 0.4063470	total: 3m 50s	remaining: 3m 50s
500:	learn: 0.4062954	total: 3m 50s	remaining: 3m 49s
501:	learn: 0.4062517	total: 3m 50s	remaining: 3m 49s
502:	learn: 0.4062090	total: 3m 51s	remaining: 3m 48s
503:	learn: 0.4061643	total: 3m 51s	remaining: 3m 48s
504:	learn: 0.4060945	total: 3m 52s	remaining: 3m 47s
505:	learn: 0.4060664	total: 3m 52s	remaining: 3m 47s
506:	learn: 0.4060023	total: 3m 53s	remaining: 3m 46s
507:	learn: 0.4059759	total:

643:	learn: 0.4002719	total: 4m 58s	remaining: 2m 45s
644:	learn: 0.4002443	total: 4m 59s	remaining: 2m 44s
645:	learn: 0.4002203	total: 4m 59s	remaining: 2m 44s
646:	learn: 0.4001711	total: 5m	remaining: 2m 43s
647:	learn: 0.4001368	total: 5m	remaining: 2m 43s
648:	learn: 0.4001207	total: 5m 1s	remaining: 2m 42s
649:	learn: 0.4000491	total: 5m 1s	remaining: 2m 42s
650:	learn: 0.4000243	total: 5m 1s	remaining: 2m 41s
651:	learn: 0.3999719	total: 5m 2s	remaining: 2m 41s
652:	learn: 0.3999549	total: 5m 2s	remaining: 2m 40s
653:	learn: 0.3998929	total: 5m 3s	remaining: 2m 40s
654:	learn: 0.3998559	total: 5m 3s	remaining: 2m 40s
655:	learn: 0.3998104	total: 5m 4s	remaining: 2m 39s
656:	learn: 0.3997685	total: 5m 4s	remaining: 2m 39s
657:	learn: 0.3997481	total: 5m 5s	remaining: 2m 38s
658:	learn: 0.3997021	total: 5m 5s	remaining: 2m 38s
659:	learn: 0.3996830	total: 5m 6s	remaining: 2m 37s
660:	learn: 0.3996344	total: 5m 6s	remaining: 2m 37s
661:	learn: 0.3996010	total: 5m 7s	remaining: 2m 

797:	learn: 0.3943471	total: 6m 11s	remaining: 1m 34s
798:	learn: 0.3943086	total: 6m 12s	remaining: 1m 33s
799:	learn: 0.3942592	total: 6m 13s	remaining: 1m 33s
800:	learn: 0.3942293	total: 6m 13s	remaining: 1m 32s
801:	learn: 0.3941911	total: 6m 14s	remaining: 1m 32s
802:	learn: 0.3941737	total: 6m 14s	remaining: 1m 31s
803:	learn: 0.3941381	total: 6m 15s	remaining: 1m 31s
804:	learn: 0.3940859	total: 6m 15s	remaining: 1m 30s
805:	learn: 0.3940305	total: 6m 16s	remaining: 1m 30s
806:	learn: 0.3939637	total: 6m 16s	remaining: 1m 30s
807:	learn: 0.3939450	total: 6m 17s	remaining: 1m 29s
808:	learn: 0.3939152	total: 6m 17s	remaining: 1m 29s
809:	learn: 0.3939077	total: 6m 18s	remaining: 1m 28s
810:	learn: 0.3938735	total: 6m 18s	remaining: 1m 28s
811:	learn: 0.3938315	total: 6m 19s	remaining: 1m 27s
812:	learn: 0.3937439	total: 6m 19s	remaining: 1m 27s
813:	learn: 0.3937111	total: 6m 20s	remaining: 1m 26s
814:	learn: 0.3936649	total: 6m 20s	remaining: 1m 26s
815:	learn: 0.3936274	total:

952:	learn: 0.3885800	total: 7m 27s	remaining: 22.1s
953:	learn: 0.3885365	total: 7m 28s	remaining: 21.6s
954:	learn: 0.3884990	total: 7m 28s	remaining: 21.1s
955:	learn: 0.3884680	total: 7m 29s	remaining: 20.7s
956:	learn: 0.3884193	total: 7m 29s	remaining: 20.2s
957:	learn: 0.3883869	total: 7m 30s	remaining: 19.7s
958:	learn: 0.3883562	total: 7m 30s	remaining: 19.3s
959:	learn: 0.3883308	total: 7m 31s	remaining: 18.8s
960:	learn: 0.3883047	total: 7m 31s	remaining: 18.3s
961:	learn: 0.3882945	total: 7m 32s	remaining: 17.9s
962:	learn: 0.3882593	total: 7m 32s	remaining: 17.4s
963:	learn: 0.3882054	total: 7m 33s	remaining: 16.9s
964:	learn: 0.3881640	total: 7m 33s	remaining: 16.4s
965:	learn: 0.3881257	total: 7m 34s	remaining: 16s
966:	learn: 0.3880844	total: 7m 34s	remaining: 15.5s
967:	learn: 0.3880685	total: 7m 35s	remaining: 15s
968:	learn: 0.3880302	total: 7m 35s	remaining: 14.6s
969:	learn: 0.3880063	total: 7m 36s	remaining: 14.1s
970:	learn: 0.3879690	total: 7m 36s	remaining: 13.

In [24]:
np.mean(cv_scores)

0.7548549727730196

In [25]:
%%time
ctb.fit(train_df_part, y_train_part,
        cat_features=cat_features_idx);

Learning rate set to 0.055755
0:	learn: 0.6642624	total: 733ms	remaining: 12m 12s
1:	learn: 0.6402777	total: 1.14s	remaining: 9m 30s
2:	learn: 0.6177301	total: 1.72s	remaining: 9m 32s
3:	learn: 0.5992438	total: 1.99s	remaining: 8m 16s
4:	learn: 0.5833098	total: 2.37s	remaining: 7m 50s
5:	learn: 0.5673575	total: 3.08s	remaining: 8m 29s
6:	learn: 0.5541217	total: 3.61s	remaining: 8m 32s
7:	learn: 0.5434691	total: 4s	remaining: 8m 15s
8:	learn: 0.5329440	total: 4.39s	remaining: 8m 3s
9:	learn: 0.5235674	total: 4.87s	remaining: 8m 1s
10:	learn: 0.5144076	total: 5.56s	remaining: 8m 19s
11:	learn: 0.5089099	total: 5.75s	remaining: 7m 53s
12:	learn: 0.5023632	total: 6.43s	remaining: 8m 8s
13:	learn: 0.4973481	total: 6.69s	remaining: 7m 51s
14:	learn: 0.4925266	total: 7.39s	remaining: 8m 5s
15:	learn: 0.4870247	total: 8.28s	remaining: 8m 29s
16:	learn: 0.4823998	total: 9.21s	remaining: 8m 52s
17:	learn: 0.4777736	total: 9.98s	remaining: 9m 4s
18:	learn: 0.4745758	total: 10.6s	remaining: 9m 8s


154:	learn: 0.4233703	total: 1m 58s	remaining: 10m 45s
155:	learn: 0.4232569	total: 1m 59s	remaining: 10m 46s
156:	learn: 0.4231737	total: 2m	remaining: 10m 46s
157:	learn: 0.4230209	total: 2m	remaining: 10m 44s
158:	learn: 0.4229357	total: 2m 1s	remaining: 10m 42s
159:	learn: 0.4229079	total: 2m 2s	remaining: 10m 40s
160:	learn: 0.4228362	total: 2m 2s	remaining: 10m 40s
161:	learn: 0.4227538	total: 2m 3s	remaining: 10m 38s
162:	learn: 0.4226196	total: 2m 4s	remaining: 10m 39s
163:	learn: 0.4225533	total: 2m 5s	remaining: 10m 37s
164:	learn: 0.4225151	total: 2m 6s	remaining: 10m 37s
165:	learn: 0.4224713	total: 2m 6s	remaining: 10m 37s
166:	learn: 0.4224243	total: 2m 7s	remaining: 10m 35s
167:	learn: 0.4223711	total: 2m 8s	remaining: 10m 37s
168:	learn: 0.4223005	total: 2m 9s	remaining: 10m 38s
169:	learn: 0.4222274	total: 2m 11s	remaining: 10m 39s
170:	learn: 0.4221704	total: 2m 12s	remaining: 10m 40s
171:	learn: 0.4221069	total: 2m 12s	remaining: 10m 40s
172:	learn: 0.4220842	total: 

306:	learn: 0.4137292	total: 4m 5s	remaining: 9m 14s
307:	learn: 0.4136800	total: 4m 6s	remaining: 9m 13s
308:	learn: 0.4136469	total: 4m 7s	remaining: 9m 12s
309:	learn: 0.4136202	total: 4m 8s	remaining: 9m 12s
310:	learn: 0.4135749	total: 4m 9s	remaining: 9m 11s
311:	learn: 0.4135586	total: 4m 10s	remaining: 9m 11s
312:	learn: 0.4135001	total: 4m 11s	remaining: 9m 10s
313:	learn: 0.4134573	total: 4m 11s	remaining: 9m 9s
314:	learn: 0.4134090	total: 4m 12s	remaining: 9m 8s
315:	learn: 0.4133753	total: 4m 12s	remaining: 9m 7s
316:	learn: 0.4133332	total: 4m 13s	remaining: 9m 6s
317:	learn: 0.4132867	total: 4m 14s	remaining: 9m 5s
318:	learn: 0.4131955	total: 4m 15s	remaining: 9m 5s
319:	learn: 0.4131769	total: 4m 16s	remaining: 9m 5s
320:	learn: 0.4131266	total: 4m 17s	remaining: 9m 5s
321:	learn: 0.4130807	total: 4m 18s	remaining: 9m 4s
322:	learn: 0.4130541	total: 4m 19s	remaining: 9m 3s
323:	learn: 0.4129759	total: 4m 20s	remaining: 9m 2s
324:	learn: 0.4128845	total: 4m 20s	remainin

459:	learn: 0.4071355	total: 5m 52s	remaining: 6m 53s
460:	learn: 0.4071135	total: 5m 52s	remaining: 6m 52s
461:	learn: 0.4070617	total: 5m 53s	remaining: 6m 51s
462:	learn: 0.4070194	total: 5m 54s	remaining: 6m 50s
463:	learn: 0.4069461	total: 5m 54s	remaining: 6m 49s
464:	learn: 0.4069093	total: 5m 55s	remaining: 6m 48s
465:	learn: 0.4068806	total: 5m 56s	remaining: 6m 48s
466:	learn: 0.4068393	total: 5m 56s	remaining: 6m 47s
467:	learn: 0.4068082	total: 5m 57s	remaining: 6m 46s
468:	learn: 0.4067776	total: 5m 57s	remaining: 6m 45s
469:	learn: 0.4067449	total: 5m 58s	remaining: 6m 44s
470:	learn: 0.4067110	total: 5m 59s	remaining: 6m 43s
471:	learn: 0.4066834	total: 5m 59s	remaining: 6m 42s
472:	learn: 0.4066599	total: 6m	remaining: 6m 41s
473:	learn: 0.4066332	total: 6m 1s	remaining: 6m 40s
474:	learn: 0.4065775	total: 6m 1s	remaining: 6m 39s
475:	learn: 0.4065530	total: 6m 2s	remaining: 6m 38s
476:	learn: 0.4064771	total: 6m 3s	remaining: 6m 38s
477:	learn: 0.4064329	total: 6m 3s	r

612:	learn: 0.4014779	total: 7m 35s	remaining: 4m 47s
613:	learn: 0.4014654	total: 7m 35s	remaining: 4m 46s
614:	learn: 0.4014335	total: 7m 36s	remaining: 4m 45s
615:	learn: 0.4014046	total: 7m 37s	remaining: 4m 44s
616:	learn: 0.4013672	total: 7m 37s	remaining: 4m 44s
617:	learn: 0.4013450	total: 7m 38s	remaining: 4m 43s
618:	learn: 0.4013032	total: 7m 39s	remaining: 4m 42s
619:	learn: 0.4012592	total: 7m 39s	remaining: 4m 41s
620:	learn: 0.4012337	total: 7m 40s	remaining: 4m 41s
621:	learn: 0.4011961	total: 7m 41s	remaining: 4m 40s
622:	learn: 0.4011749	total: 7m 41s	remaining: 4m 39s
623:	learn: 0.4011239	total: 7m 42s	remaining: 4m 38s
624:	learn: 0.4010941	total: 7m 42s	remaining: 4m 37s
625:	learn: 0.4010544	total: 7m 43s	remaining: 4m 36s
626:	learn: 0.4010170	total: 7m 44s	remaining: 4m 36s
627:	learn: 0.4009723	total: 7m 44s	remaining: 4m 35s
628:	learn: 0.4009235	total: 7m 45s	remaining: 4m 34s
629:	learn: 0.4008712	total: 7m 45s	remaining: 4m 33s
630:	learn: 0.4008458	total:

766:	learn: 0.3961776	total: 9m 19s	remaining: 2m 49s
767:	learn: 0.3961522	total: 9m 19s	remaining: 2m 49s
768:	learn: 0.3961311	total: 9m 20s	remaining: 2m 48s
769:	learn: 0.3961050	total: 9m 21s	remaining: 2m 47s
770:	learn: 0.3960914	total: 9m 21s	remaining: 2m 46s
771:	learn: 0.3960720	total: 9m 22s	remaining: 2m 46s
772:	learn: 0.3960492	total: 9m 22s	remaining: 2m 45s
773:	learn: 0.3960261	total: 9m 23s	remaining: 2m 44s
774:	learn: 0.3959819	total: 9m 24s	remaining: 2m 43s
775:	learn: 0.3959238	total: 9m 24s	remaining: 2m 43s
776:	learn: 0.3959052	total: 9m 25s	remaining: 2m 42s
777:	learn: 0.3958652	total: 9m 26s	remaining: 2m 41s
778:	learn: 0.3958274	total: 9m 26s	remaining: 2m 40s
779:	learn: 0.3958150	total: 9m 27s	remaining: 2m 40s
780:	learn: 0.3957893	total: 9m 28s	remaining: 2m 39s
781:	learn: 0.3957497	total: 9m 28s	remaining: 2m 38s
782:	learn: 0.3956989	total: 9m 29s	remaining: 2m 37s
783:	learn: 0.3956706	total: 9m 30s	remaining: 2m 37s
784:	learn: 0.3956492	total:

918:	learn: 0.3911570	total: 11m 4s	remaining: 58.5s
919:	learn: 0.3911491	total: 11m 4s	remaining: 57.8s
920:	learn: 0.3911355	total: 11m 5s	remaining: 57.1s
921:	learn: 0.3911173	total: 11m 5s	remaining: 56.3s
922:	learn: 0.3911050	total: 11m 6s	remaining: 55.6s
923:	learn: 0.3910811	total: 11m 7s	remaining: 54.9s
924:	learn: 0.3910519	total: 11m 7s	remaining: 54.2s
925:	learn: 0.3910337	total: 11m 8s	remaining: 53.4s
926:	learn: 0.3909917	total: 11m 9s	remaining: 52.7s
927:	learn: 0.3909746	total: 11m 9s	remaining: 52s
928:	learn: 0.3909350	total: 11m 10s	remaining: 51.3s
929:	learn: 0.3909049	total: 11m 11s	remaining: 50.5s
930:	learn: 0.3908805	total: 11m 11s	remaining: 49.8s
931:	learn: 0.3908693	total: 11m 12s	remaining: 49.1s
932:	learn: 0.3908391	total: 11m 13s	remaining: 48.3s
933:	learn: 0.3907978	total: 11m 13s	remaining: 47.6s
934:	learn: 0.3907419	total: 11m 14s	remaining: 46.9s
935:	learn: 0.3907105	total: 11m 15s	remaining: 46.2s
936:	learn: 0.3907032	total: 11m 15s	rem

In [26]:
roc_auc_score(y_valid, ctb.predict_proba(valid_df)[:, 1])

0.7665412163695575

In [27]:
ctb.fit(train_df, y, cat_features=cat_features_idx)

Learning rate set to 0.061677
0:	learn: 0.6601661	total: 990ms	remaining: 16m 28s
1:	learn: 0.6341576	total: 1.34s	remaining: 11m 9s
2:	learn: 0.6102753	total: 2.38s	remaining: 13m 9s
3:	learn: 0.5885690	total: 3.62s	remaining: 15m 1s
4:	learn: 0.5708779	total: 5.07s	remaining: 16m 48s
5:	learn: 0.5561537	total: 5.54s	remaining: 15m 18s
6:	learn: 0.5414672	total: 6.58s	remaining: 15m 33s
7:	learn: 0.5292089	total: 7.37s	remaining: 15m 13s
8:	learn: 0.5184327	total: 8.21s	remaining: 15m 3s
9:	learn: 0.5094093	total: 9.15s	remaining: 15m 5s
10:	learn: 0.5025585	total: 9.56s	remaining: 14m 19s
11:	learn: 0.4954555	total: 10.5s	remaining: 14m 25s
12:	learn: 0.4888893	total: 11.2s	remaining: 14m 13s
13:	learn: 0.4827546	total: 12.1s	remaining: 14m 12s
14:	learn: 0.4778954	total: 12.9s	remaining: 14m 9s
15:	learn: 0.4732320	total: 14s	remaining: 14m 18s
16:	learn: 0.4700418	total: 14.8s	remaining: 14m 15s
17:	learn: 0.4663376	total: 15.7s	remaining: 14m 14s
18:	learn: 0.4634265	total: 16.6s	

153:	learn: 0.4189552	total: 2m 29s	remaining: 13m 42s
154:	learn: 0.4188793	total: 2m 30s	remaining: 13m 42s
155:	learn: 0.4188162	total: 2m 31s	remaining: 13m 41s
156:	learn: 0.4186901	total: 2m 32s	remaining: 13m 40s
157:	learn: 0.4186120	total: 2m 33s	remaining: 13m 39s
158:	learn: 0.4186062	total: 2m 34s	remaining: 13m 36s
159:	learn: 0.4185589	total: 2m 35s	remaining: 13m 34s
160:	learn: 0.4184532	total: 2m 35s	remaining: 13m 32s
161:	learn: 0.4183829	total: 2m 36s	remaining: 13m 29s
162:	learn: 0.4183307	total: 2m 37s	remaining: 13m 28s
163:	learn: 0.4182605	total: 2m 38s	remaining: 13m 26s
164:	learn: 0.4182092	total: 2m 39s	remaining: 13m 28s
165:	learn: 0.4181465	total: 2m 40s	remaining: 13m 28s
166:	learn: 0.4180309	total: 2m 42s	remaining: 13m 28s
167:	learn: 0.4179364	total: 2m 42s	remaining: 13m 26s
168:	learn: 0.4179068	total: 2m 43s	remaining: 13m 24s
169:	learn: 0.4178734	total: 2m 44s	remaining: 13m 23s
170:	learn: 0.4178251	total: 2m 45s	remaining: 13m 21s
171:	learn

304:	learn: 0.4103265	total: 4m 45s	remaining: 10m 51s
305:	learn: 0.4102986	total: 4m 46s	remaining: 10m 50s
306:	learn: 0.4102540	total: 4m 47s	remaining: 10m 49s
307:	learn: 0.4102148	total: 4m 48s	remaining: 10m 48s
308:	learn: 0.4101603	total: 4m 49s	remaining: 10m 47s
309:	learn: 0.4101016	total: 4m 50s	remaining: 10m 45s
310:	learn: 0.4100417	total: 4m 51s	remaining: 10m 44s
311:	learn: 0.4099968	total: 4m 51s	remaining: 10m 43s
312:	learn: 0.4099313	total: 4m 52s	remaining: 10m 42s
313:	learn: 0.4098980	total: 4m 53s	remaining: 10m 41s
314:	learn: 0.4098478	total: 4m 54s	remaining: 10m 40s
315:	learn: 0.4097770	total: 4m 55s	remaining: 10m 39s
316:	learn: 0.4097272	total: 4m 56s	remaining: 10m 38s
317:	learn: 0.4096468	total: 4m 57s	remaining: 10m 37s
318:	learn: 0.4096153	total: 4m 58s	remaining: 10m 36s
319:	learn: 0.4095756	total: 4m 58s	remaining: 10m 34s
320:	learn: 0.4095527	total: 4m 59s	remaining: 10m 33s
321:	learn: 0.4095248	total: 5m	remaining: 10m 32s
322:	learn: 0.

457:	learn: 0.4041808	total: 7m 11s	remaining: 8m 30s
458:	learn: 0.4041568	total: 7m 12s	remaining: 8m 30s
459:	learn: 0.4041242	total: 7m 14s	remaining: 8m 29s
460:	learn: 0.4040876	total: 7m 15s	remaining: 8m 28s
461:	learn: 0.4040632	total: 7m 16s	remaining: 8m 27s
462:	learn: 0.4040352	total: 7m 17s	remaining: 8m 27s
463:	learn: 0.4039942	total: 7m 19s	remaining: 8m 27s
464:	learn: 0.4038622	total: 7m 20s	remaining: 8m 27s
465:	learn: 0.4038062	total: 7m 22s	remaining: 8m 26s
466:	learn: 0.4037803	total: 7m 23s	remaining: 8m 26s
467:	learn: 0.4037475	total: 7m 24s	remaining: 8m 25s
468:	learn: 0.4037006	total: 7m 25s	remaining: 8m 24s
469:	learn: 0.4036699	total: 7m 26s	remaining: 8m 23s
470:	learn: 0.4036522	total: 7m 27s	remaining: 8m 22s
471:	learn: 0.4036133	total: 7m 28s	remaining: 8m 21s
472:	learn: 0.4035759	total: 7m 30s	remaining: 8m 21s
473:	learn: 0.4035244	total: 7m 32s	remaining: 8m 21s
474:	learn: 0.4034564	total: 7m 33s	remaining: 8m 21s
475:	learn: 0.4033941	total:

610:	learn: 0.3990462	total: 10m 10s	remaining: 6m 28s
611:	learn: 0.3989876	total: 10m 11s	remaining: 6m 27s
612:	learn: 0.3989729	total: 10m 12s	remaining: 6m 26s
613:	learn: 0.3989521	total: 10m 13s	remaining: 6m 25s
614:	learn: 0.3989324	total: 10m 14s	remaining: 6m 24s
615:	learn: 0.3989087	total: 10m 16s	remaining: 6m 24s
616:	learn: 0.3988627	total: 10m 17s	remaining: 6m 23s
617:	learn: 0.3988439	total: 10m 18s	remaining: 6m 22s
618:	learn: 0.3987943	total: 10m 19s	remaining: 6m 21s
619:	learn: 0.3987458	total: 10m 20s	remaining: 6m 20s
620:	learn: 0.3987132	total: 10m 21s	remaining: 6m 19s
621:	learn: 0.3986907	total: 10m 23s	remaining: 6m 18s
622:	learn: 0.3986687	total: 10m 24s	remaining: 6m 18s
623:	learn: 0.3986530	total: 10m 26s	remaining: 6m 17s
624:	learn: 0.3986203	total: 10m 27s	remaining: 6m 16s
625:	learn: 0.3985726	total: 10m 28s	remaining: 6m 15s
626:	learn: 0.3985584	total: 10m 30s	remaining: 6m 14s
627:	learn: 0.3985283	total: 10m 31s	remaining: 6m 14s
628:	learn

761:	learn: 0.3943953	total: 13m 21s	remaining: 4m 10s
762:	learn: 0.3943853	total: 13m 22s	remaining: 4m 9s
763:	learn: 0.3943598	total: 13m 23s	remaining: 4m 8s
764:	learn: 0.3943463	total: 13m 24s	remaining: 4m 7s
765:	learn: 0.3943188	total: 13m 25s	remaining: 4m 6s
766:	learn: 0.3942984	total: 13m 27s	remaining: 4m 5s
767:	learn: 0.3942823	total: 13m 29s	remaining: 4m 4s
768:	learn: 0.3942452	total: 13m 30s	remaining: 4m 3s
769:	learn: 0.3942270	total: 13m 32s	remaining: 4m 2s
770:	learn: 0.3942049	total: 13m 34s	remaining: 4m 1s
771:	learn: 0.3941948	total: 13m 35s	remaining: 4m
772:	learn: 0.3941697	total: 13m 36s	remaining: 3m 59s
773:	learn: 0.3941328	total: 13m 37s	remaining: 3m 58s
774:	learn: 0.3940891	total: 13m 38s	remaining: 3m 57s
775:	learn: 0.3940765	total: 13m 39s	remaining: 3m 56s
776:	learn: 0.3940611	total: 13m 40s	remaining: 3m 55s
777:	learn: 0.3940227	total: 13m 41s	remaining: 3m 54s
778:	learn: 0.3939985	total: 13m 42s	remaining: 3m 53s
779:	learn: 0.3939718	t

912:	learn: 0.3901476	total: 16m 15s	remaining: 1m 32s
913:	learn: 0.3901347	total: 16m 16s	remaining: 1m 31s
914:	learn: 0.3901091	total: 16m 17s	remaining: 1m 30s
915:	learn: 0.3900826	total: 16m 18s	remaining: 1m 29s
916:	learn: 0.3900732	total: 16m 19s	remaining: 1m 28s
917:	learn: 0.3900478	total: 16m 20s	remaining: 1m 27s
918:	learn: 0.3900096	total: 16m 21s	remaining: 1m 26s
919:	learn: 0.3899833	total: 16m 22s	remaining: 1m 25s
920:	learn: 0.3899365	total: 16m 23s	remaining: 1m 24s
921:	learn: 0.3899251	total: 16m 24s	remaining: 1m 23s
922:	learn: 0.3898775	total: 16m 25s	remaining: 1m 22s
923:	learn: 0.3898429	total: 16m 26s	remaining: 1m 21s
924:	learn: 0.3898229	total: 16m 27s	remaining: 1m 20s
925:	learn: 0.3898011	total: 16m 28s	remaining: 1m 19s
926:	learn: 0.3897788	total: 16m 30s	remaining: 1m 17s
927:	learn: 0.3897455	total: 16m 31s	remaining: 1m 16s
928:	learn: 0.3897201	total: 16m 32s	remaining: 1m 15s
929:	learn: 0.3896988	total: 16m 34s	remaining: 1m 14s
930:	learn

CatBoostError: Invalid cat_features[6] = 8 value: must be < 8.

In [31]:
test['route'] = test.apply(lambda x: x['Origin'] + '-' + x['Dest'], 1)

In [33]:
ctb_test_pred = ctb.predict_proba(test)[:, 1]

In [34]:
pd.Series(ctb_test_pred, name='dep_delayed_15min').to_csv('submit_ctb.csv', 
                                                          index_label='id', 
                                                          header=True)

In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
Month                100000 non-null object
DayofMonth           100000 non-null object
DayOfWeek            100000 non-null object
DepTime              100000 non-null int64
UniqueCarrier        100000 non-null object
Origin               100000 non-null object
Dest                 100000 non-null object
Distance             100000 non-null int64
dep_delayed_15min    100000 non-null object
dtypes: int64(2), object(7)
memory usage: 6.9+ MB


In [8]:
train['route'] = train.apply(lambda x: x['Origin'] + '-' + x['Dest'], 1)

In [4]:
dummies = ['Month', 'DayofMonth', 'DayOfWeek', 'UniqueCarrier', 'route']
train_dummies = pd.get_dummies(train[dummies])
train_dummies.head()

,Month_c-1,Month_c-10,Month_c-11,Month_c-12,Month_c-2,Month_c-3,Month_c-4,Month_c-5,Month_c-6,Month_c-7,...,route_XNA-IAH,route_XNA-LAX,route_XNA-LGA,route_XNA-ORD,route_XNA-SLC,route_YAK-CDV,route_YAK-JNU,route_YUM-IPL,route_YUM-LAX,route_YUM-PHX
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
train.head()

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min,route
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N,ATL-DFW
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N,PIT-MCO
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N,RDU-CLE
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N,DEN-MEM
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y,MDW-OMA


In [6]:
X = train.drop(['dep_delayed_15min'], axis=1)

In [7]:
y = train['dep_delayed_15min'].map({'Y': 1, 'N': 0})

In [8]:
num_var = ['DepTime', 'Distance']
X = pd.concat([train[num_var], train_dummies], axis=1)
X.head()

,DepTime,Distance,Month_c-1,Month_c-10,Month_c-11,Month_c-12,Month_c-2,Month_c-3,Month_c-4,Month_c-5,...,route_XNA-IAH,route_XNA-LAX,route_XNA-LGA,route_XNA-ORD,route_XNA-SLC,route_YAK-CDV,route_YAK-JNU,route_YUM-IPL,route_YUM-LAX,route_YUM-PHX
0,1934,732,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1548,834,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1422,416,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1015,872,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1828,423,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
split = int(X.shape[0] * 0.7)

In [10]:
X_train = X.iloc[:split, :]
X_valid = X.iloc[split:, :]

In [11]:
y_train, y_valid = y.iloc[:split], y.iloc[split:]

In [12]:
%%time
xgb = XGBClassifier()
xgb.fit(X_train, y_train)

MemoryError: 

In [ ]:
xgb_valid_pred = xgb.predict_proba(X_valid)[:, 1]
roc_auc_score(y_valid, xgb_valid_pred)